# Step 1.

# Data Loading

In [1]:
#Import libraries

import random
import scanpy as sc
import scvi
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import squidpy as sq
from pathlib import Path

random.seed(10)

Global seed set to 0
/data/leuven/338/vsc33876/miniconda3/envs/python38/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/data/leuven/338/vsc33876/miniconda3/envs/python38/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
#Change working directory for consistency

os.chdir('/vsc-hard-mounts/leuven-data/338/vsc33876/IBP_Stein_2022/Data/')

In [3]:
#Read the full RNA seq data

adata_full = sc.read_h5ad('IBP_file.h5ad')

In [ ]:
#Create a new annData object with the cell type subcluster, if the annotations are already present

adata = adata_full[adata_full.obs.celltype == 'ODC']

# Step 2
# Preprocessing and Visualization


In [ ]:
#Check for mitochondria genes

adata.var['mt'] = adata.var.index.str.startswith('MT-')
adata.var['mt']

In [ ]:
#Check for the ribosomal genes

ribo_url = "http://software.broadinstitute.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt"
ribo_genes = pd.read_table(ribo_url, skiprows=2, header = None)
adata.var['ribo'] = adata.var_names.isin(ribo_genes[0].values)
adata.var['ribo']

In [ ]:
#Calculate QC metrics, controlling for ribo and mito

sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)

In [ ]:
#Check for the distribution of the genes. 

sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'], 
             jitter=0.4, multi_panel=True)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 4))
sns.distplot(
    adata.obs["total_counts"],
    kde=False,
    ax=axs[0],
)
sns.distplot(
    adata.obs["n_genes_by_counts"],
    kde=False,
    bins=60,
    ax=axs[1],
)

In [ ]:
#Check the lowest and highest number of cells

adata.var.sort_values('n_cells_by_counts')

In [ ]:
#Check the lowest and highest number of genes expressed

adata.obs.sort_values('n_genes_by_counts')

In [ ]:
#Filter the dataset rationally

sc.pp.filter_cells(adata, min_genes=500)
sc.pp.filter_genes(adata, min_cells=20)

In [ ]:
#Normalize every cell to 10,000 UMI

sc.pp.normalize_total(adata, target_sum=1e4) 
adata.X.sum(axis = 1)

In [ ]:
#Change to log counts

sc.pp.log1p(adata)
adata.X.sum(axis = 1)

In [ ]:
#Check the distribution of the data

sns.countplot(x = adata.obs.Diagnosis)
sns.countplot(x = adata.obs.Sex)

In [ ]:
#Save the raw data in the anndata object

adata.raw = adata

# Step 3
# Clustering

In [ ]:
#Choose highly variable genes, based on the size of your data

sc.pp.highly_variable_genes(adata, n_top_genes = 5000)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
#Add variable genes to anndata object

adata = adata[:, adata.var.highly_variable]

In [ ]:
#Regress out what can bring variability to your data.

sc.pp.regress_out(adata, ['total_counts', 'pct_counts_ribo'])

In [ ]:
sc.pp.scale(adata, max_value=10)

In [ ]:
#Run PCA analysis on the data

sc.tl.pca(adata)

In [ ]:
#Plot all PCs 

sc.pl.pca_variance_ratio(adata, log=True, n_pcs = 50) 

In [ ]:
#Choose number of PCAs to base your clusterings on

sc.pp.neighbors(adata, n_pcs = 35)

In [ ]:
#Run UMAP on the obtained data

sc.pl.umap(adata)

In [ ]:
#Check if there are potential batch effects, can be from the samples or from the diffferent runs.

sc.pl.umap(adata, color= ['Sample.ID'], ncols = 2)

#Clearly there is a batch effect in the samples. Have to correct for that. 

# Step 4
# Batch Correction Samples with Harmony

In [ ]:
from harmony import harmonize

In [ ]:
Z = harmonize(adata.obsm['X_pca'], adata.obs, batch_key = 'Sample.ID')
adata.obsm['X_harmony'] = Z

In [ ]:
#Choose amount of PCAs under harmony PCA analysis

sc.pp.neighbors(adata, n_pcs = 35, use_rep = 'X_harmony')

In [ ]:
sc.tl.umap(adata, init_pos = 'X_harmony')

In [ ]:
#Empty mapping

sc.pl.umap(adata) 

In [ ]:
#Use leiden clustering (substituted for louvain method) to cluster obtained UMAP

sc.tl.leiden(adata, resolution = 1.0, key_added='leiden1') #0-1 from less cluters to more, can play with numbers
sc.tl.leiden(adata, resolution = 0.1, key_added='leiden01')
sc.tl.leiden(adata, resolution = 0.4, key_added='leiden04')
sc.tl.leiden(adata, resolution = 0.5, key_added='leiden05') 
sc.tl.leiden(adata, resolution = 0.7, key_added='leiden07')

In [ ]:
#Plot UMAPS and choose the clusterings that fit your data the most. Y

sc.pl.umap(adata, color= ['Sex', 'Age', 'Diagnosis', 'leiden1', 'leiden01' ,'leiden04', 'leiden05', 'leiden07'], ncols = 2)

In [ ]:
#Save the anndata object

adata.write("Oligo_harmony.h5ad")

# Step 5
# Cross-reference of suspicious clusters with Allen Brain Atlas marker genes

In [ ]:
#Plot the clusters against the variable of interest 

sc.pl.umap(
    adata,
    color=["Diagnosis", "leiden04"],
    ncols=2,
    frameon=False,
)

#Some previously annotated clusters might be annotated incorrectly. 

In [ ]:
#Allen Brain dataset was downloaded from the 10x RNA seq from the Allen Brain website
#https://portal.brain-map.org/atlases-and-data/rnaseq

allen = pd.read_csv('trimmed_means MTG.csv')
allen

In [ ]:
#Reorder Allen data for convinience, look up the columns with the cell types of interest

allen = allen.reindex(sorted(allen.columns), axis=1)
filter_oligo = [col for col in allen if 'Oligo' in col]

In [ ]:
#Create a new dataframe just with the cell types of interest and their relative expression. Save as csv for convinience. 

filtered_oligo = allen[['gene_symbol', 'Oligo_1', 'Oligo_2', 'Oligo_4', 'Oligo_3']]
maxvalue = filtered_oligo.max(axis=1)
filtered_oligo['Oligo'] = maxvalue
filtered_oligo.drop(['Oligo_1', 'Oligo_2', 'Oligo_4', 'Oligo_3'], inplace=False, axis=1).loc[filtered_oligo['Oligo'] > 0].sort_values("Oligo", ascending = False)['feature'].to_csv('/vsc-hard-mounts/leuven-data/338/vsc33876/IBP_Stein_2022/Data/oligo.csv', index=False)

In [ ]:
#Score the genes from the Allen Brain atlas against your data

sc.tl.score_genes(adata, oligo['gene_symbol'], score_name='Oligo MG')

In [ ]:
#Now check UMAP. If expression in certain clusters is less than in the rest of them, they were probably annotated incorrectly.

sc.pl.umap(
    adata,
    color=["leiden04", 'Oligo MG'],
    ncols=2,
    frameon=False,
    vmin = 'p1.5', vmax = 'p99'
)

### Repeat step 5 for other cell types.

In [ ]:
#Plot all cell types to find the right annotation for your misannotated cluster. 

sc.pl.umap(adata, color=[ 'Oligo MG', 'astro MG', 'l5 MG', 'l6 MG', 'l2 MG', 'chandelier MG', 'vlmc MG', 'opc MG', 'micro MG', 'endo MG'], 
           ncols = 2, 
           legend_loc = 'on data',
           frameon=True,
           )


Filter the data if needed. Change the annotation of the data if needed. If those aforementioned actions are performed, Step 3 should be reperated. 

In this example, Oligodendrocytes were annotated correctly, according to the Allen Brain marker genes. 

# Step 6
# Ranked Genes

In [ ]:
#Dataframe with ranked genes by clusters. Can be applied for other rankings too.
cluster_ranked_genes_l04 = pd.DataFrame()
sc.tl.rank_genes_groups(adata, groupby="leiden04", use_raw=True)
sc.pl.rank_genes_groups(adata, n_genes=10, sharey=False)
for i in range(8):
    cluster_ranked_genes_l04[i] = sc.get.rank_genes_groups_df(adata, group=str(i))[['names']]

In [ ]:
#Save for GO analysis
cluster_ranked_genes_l04.to_csv('/vsc-hard-mounts/leuven-data/338/vsc33876/IBP_Stein_2022/Data/go_oligo/clustersgenes07.csv')

In [ ]:
#Dataframe with ranked genes by Diagnosis
cluster_ranked_genes_Diag = pd.DataFrame()
sc.tl.rank_genes_groups(adata, groupby="Diagnosis", use_raw=True)
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False)
cluster_ranked_genes_Diag = sc.get.rank_genes_groups_df(adata, group=None)

In [ ]:
#Save AD and control genes
cluster_ranked_genes_Diag[cluster_ranked_genes_Diag['group'] == "AD"]['names'].to_csv('/vsc-hard-mounts/leuven-data/338/vsc33876/IBP_Stein_2022/Data/go_oligo/ad.csv')
cluster_ranked_genes_Diag[cluster_ranked_genes_Diag['group'] == "Control"]['names'].to_csv('/vsc-hard-mounts/leuven-data/338/vsc33876/IBP_Stein_2022/Data/go_oligo/control.csv')

In [ ]:
#Dataframe with ranked genes by Sex
cluster_ranked_genes_Sex = pd.DataFrame()
sc.tl.rank_genes_groups(adata, groupby="Sex", use_raw=True)
sc.pl.rank_genes_groups(adata, n_genes=7, sharey=False)
cluster_ranked_genes_Sex = sc.get.rank_genes_groups_df(adata, group= None)

In [ ]:
#Save sex
cluster_ranked_genes_Sex[cluster_ranked_genes_Sex['group'] == "F"]['names'].to_csv('/vsc-hard-mounts/leuven-data/338/vsc33876/IBP_Stein_2022/Data/go_oligo/f.csv')
cluster_ranked_genes_Sex[cluster_ranked_genes_Sex['group'] == "M"]['names'].to_csv('/vsc-hard-mounts/leuven-data/338/vsc33876/IBP_Stein_2022/Data/go_oligo/m.csv')

# Step 7 
# GO analysis with GOrilla and G-profiler

In [ ]:
#AD
http://cbl-gorilla.cs.technion.ac.il/GOrilla/kkacagh5/GOResults.html
    

https://biit.cs.ut.ee/gprofiler/gost?organism=hsapiens&query=XIST%0AERBIN%0ANEAT1%0APTPRM%0AFKBP5%0ACHORDC1%0AXPO1%0AAC063952.1%0ACDKL1%0AMYRIP%0AFXR1%0APTGES3%0AHSPA1A%0AMALAT1%0APAPOLA%0ALURAP1L-AS1%0AHSPH1%0ALINC00862%0APDS5B%0APCDH9-AS2%0APCA3%0AKDM6A%0AAPOLD1%0APPFIBP2%0ASLC38A2%0AZDHHC11B%0APRKX%0ASTIP1%0AAL139294.1%0AAGMO%0ASORT1%0ADNAJB1%0ANUP210L%0AABCA6%0AHSPA4L%0AMSI2%0APRKCA%0APPP3CB-AS1%0AITPR2%0AKANSL1L%0AAC009063.2%0AERC2%0ATMEM98%0ADIP2C%0ARPL37A%0AMET%0AAC021660.3%0ASORBS1%0ADNAJA1%0ASLCO1A2%0ACDH13%0ACLMN%0ASEMA4B%0API4KAP1%0AITPRID2%0AHSP90AA1%0AATP1B3%0AATG4B%0AUTP23%0AZNF706%0ASLC6A15%0AEPB41L4B%0AAC106869.1%0AIRAK2%0AFANCC%0APLA2G4C%0AQKI%0AANKUB1%0AADAM19%0AANLN%0AATE1-AS1%0AZCCHC2%0AAC104461.1%0AHIBCH%0ANEO1%0AHNRNPA2B1%0AC17orf67%0ACCDC59%0AMIB1%0AC9orf153%0AZNF638%0ANUFIP2%0ARGS7%0AAC104806.2%0AAL160254.1%0AUBE2D3%0ALDLRAD3%0ATPD52L1%0AALDH3A2%0AMAP4K5%0AARHGAP5%0ADNAJB6%0ACARS%0ASPATS2L%0AGCNT1%0AADAMTSL3%0ASLC6A6%0AGUSBP9.1%0ANKAIN1%0AJPX%0AELF2%0AEFR3B%0APPM1K%0AHNRNPU%0AARHGEF7%0APTPN13%0ATLE4%0AGPC5%0ARAMP1%0AAP002518.2%0ALDB2%0AGLRA2%0ATSIX%0ACDH20%0ABCL6%0ANCOA7%0ASGK3%0AIRS2%0AADGRB3%0ACR1%0AHERC2P2%0ACNTN3%0ANFIB%0ALHFPL2%0APLOD3%0ACLIP4%0ASHC2%0ASSBP2%0ATMPRSS5%0AADAM12%0AHSPD1%0APEX5L-AS2%0AAC012358.3%0ASELENOP%0APTP4A2%0ACHSY3%0ASFMBT2%0AAC093879.1%0AAP003469.4%0ABANP%0AMORF4L2%0AMIR29B2CHG%0AWAC%0AHERC4%0AAC090518.1%0APDS5A%0AMAPT%0ALRRC1%0ARHBDL3%0AKAZN%0AMAP2K4%0ABAALC-AS1%0AMKLN1%0AZNF565%0ALCORL%0AJAKMIP3%0ALINC00513%0APCGF5%0AGIPR%0APNISR%0ACAMSAP2%0APRKD3%0AFUBP1%0AAEBP2%0ANDUFAF6%0APPP2R2A%0AN4BP2L2%0ACADM2%0ALINC01470%0APMP2%0AMAK%0ABNIP3L%0AAC023590.1%0AMUC20-OT1%0ALUCAT1%0ALINGO1%0AARRB1%0ADDX5%0AENO4%0ASTARD13-AS%0AARHGAP17%0ASEM1%0AHILS1%0ACDH12%0ASUGT1%0ASPINT2%0AADARB2-AS1%0ACACYBP%0AABHD3%0AKMT2E%0ACDK19%0ACCT4%0AHNRNPH3%0ADPP6%0AAL139352.1%0ADGKH%0ADYRK1A%0ALINC01138%0ARHOQ%0AUBL3%0ANRG2%0ACHDH%0ASPEN%0AAL117381.1%0APDE5A%0ASTAT3%0APDXDC1%0ACAB39L%0ADDX3X%0AFGFR1OP2%0ARELN%0ACLK1%0AANKRD36%0AUBE2B%0AAC012414.5%0AMAT2A%0AZFPM2%0ANFKBIZ%0AINTS6%0ANTM%0AHSPE1-MOB4%0AJDP2%0ASNAP23%0ASNX31%0ABAZ1A%0ACRIM1%0AHP1BP3%0AEMC10%0AUHRF1%0ATMC8%0ATMEM65%0APICALM%0APIM3%0ACOPA%0AFMO5%0AVPS13C%0AUSP54%0ARRAGD%0APGAP1%0AHSP90AB1%0AKLHL3%0ALSS%0ACREBRF%0AAMER2%0ACCP110%0AMATN2%0AJADE2%0ARAB1A%0AKIZ-AS1%0APDE4DIP%0ANDUFS2%0AST13%0AAC015923.1%0AARHGAP12%0ACDK13%0AADGRL3%0ANDRG1%0AAC009090.4%0AAC022146.2%0ACCT5%0AFAM222A-AS1%0ACAV1%0APPA2%0AADIPOR2%0APCNX1%0ATMOD2%0AGPR135%0AMAN2A1%0AATP11A%0ASMARCA2%0AZNF483%0ACTBP2%0AETV5%0AMPHOSPH8%0AAP000462.1%0AEEPD1%0APHIP%0AAC093865.1%0AUSP32%0ATANC2%0ACERS6%0ANPTX2%0APWWP2A%0ACCT3%0ATMEM273%0ARHOBTB3%0AEIF5%0AZDHHC11%0ANAA16%0AAHSA1%0APHF21B%0APLEKHH2%0AYARS%0ADPYD-AS1%0APPP4R4%0ASRSF7%0ALURAP1L%0ADDX6%0AAP001021.2%0APTPRJ%0ACCDC62%0ATTBK2%0ASLC39A11%0APDK4%0ASAFB2%0AHSPA4%0ATSEN15%0APPP2R5A%0AAL359762.3%0ALANCL1-AS1%0ASNCA%0ABCL2%0APDE1C%0ADPY19L1%0ASP110%0APDE7A%0ASNHG14%0ACUTALP%0AHBP1%0AIPO9-AS1%0ALGR4%0AUSP9X%0AAP002370.2%0ATSC22D1%0AHACD1%0AKCNIP3%0AMAPK1IP1L%0ASLK%0AMEST%0AOGA%0APHC2%0AVSIR%0AAC006994.2%0AANAPC16%0ACCDC168%0ACOL10A1%0AGLS%0AMYH1%0ARASA3%0AMPP5%0AZNF652%0AAL512625.3%0AFBXO2%0AELK3%0ASLC2A13%0ATARS%0ANCOA1%0AAL078581.3%0AKIZ%0ACSMD1%0AABHD15-AS1%0APACS2%0ATNKS%0AGAS6%0AAC010618.3%0ADEPDC1B%0AZSWIM6%0AP3H2%0AITGA11%0AAL683813.1%0AGCNA%0APHF21A%0AARRDC3%0APCDH9-AS4%0ASRPRB%0AC9orf84%0AAC008267.3%0AHDAC4%0ALINC01476%0ATMEM131L%0AAL022238.2%0ADIP2B%0APDLIM5%0ARASA1%0AEMSY%0AUBA6%0ASYT1%0AAL138881.1%0AHPRT1%0AAC018797.2%0ARPS6KA3%0AITPR1%0AKANK2%0AHEY2%0ASTX6%0ADNAJC1%0AAC245297.1%0AIVNS1ABP%0ALRRTM3%0ARAB11A%0AAL031846.2%0ABCL2L1%0ASERHL2%0AENSA%0AZFX%0AAC091180.5%0AMBOAT2%0AEFNB2%0ATCAIM%0AAC098588.2%0AAC125613.1%0ATIAM2%0AMPDZ%0ACDH19%0APPP4R2%0ASEMA5A%0ACAMK2D%0ADNM3%0ABARD1%0AKIAA0556%0AAC023509.6%0AALDH2%0APSIP1%0ALYRM2%0ASTK38%0ADIAPH1%0ALINC01098%0ASPSB1%0ABAG3%0APREX1%0ASPRED1%0AOR7C1%0ALHFPL3%0ANOVA1%0AFOXO1%0AKCNQ1%0ATHUMPD3-AS1%0AEYA4%0APHACTR3%0ANTRK2%0ACXCR4%0ATBCD%0AIGF1R%0ASLC26A3%0AAC107067.1%0AAC116903.2%0ADGKB%0ASIPA1L2%0ARAB9A%0AAP003086.2%0ASSBP1%0AHIST1H2AC%0ARYBP%0API16%0AAL590006.1%0AMAST4%0AAC010976.1%0AC2orf69%0ARELA%0ARAB11FIP2%0ACEMIP%0AAL157938.2%0ASDHA%0APYGL%0AUSPL1%0ABBX%0AAL513327.1%0AZNF595%0ATMEM26%0ADCC%0AAC012184.4%0APAK2%0APITPNC1%0APDE2A%0APURB%0AHTATIP2%0ARNF169%0AAC016831.6%0ADLGAP1%0AGRB10%0AAC098617.1%0AYOD1%0AMYH11%0AKMT2C%0APHC3%0AKLHL24%0ARFX3%0AAC009554.1%0AGIPC2%0AAC058791.1%0ASHMT1%0ADDRGK1%0ALINC01099%0AGIGYF2%0ALRRC37A16P%0ACENPC%0AADAM10%0ASPATA22%0ANRG3%0APPM1E%0APRMT3%0AHERC3%0ADNAJC21%0AMLLT10%0AAC005261.5%0ARTN4%0AMPP6%0APAFAH1B1%0AAL591686.2%0ASH3RF3&ordered=true&all_results=false&no_iea=false&combined=false&measure_underrepresentation=false&domain_scope=annotated&significance_threshold_method=g_SCS&user_threshold=0.05&numeric_namespace=ENTREZGENE_ACC&sources=GO:MF,GO:CC,GO:BP,KEGG,TF,REAC,MIRNA,HPA,CORUM,HP,WP&background=&no_evidences=false

In [ ]:
#Control
http://cbl-gorilla.cs.technion.ac.il/GOrilla/kzm8fic1/GOResults.html
    

https://biit.cs.ut.ee/gprofiler/gost?organism=hsapiens&query=PPP2R2B%0AUTY%0ANLGN4Y%0ASIK3%0ACARNS1%0ATTLL7%0AUNC5C%0AAK5%0AANOS2P%0APLP1%0AMBP%0AGLDN%0ASLC24A2%0ACTNND2%0ALDB3%0AKIRREL3%0ATP53TG5%0ASCD%0AGREB1L%0AOPALIN%0ANRXN3%0ALINC00240%0ATTTY14%0ACNP%0ACNTNAP4%0ARNASE1%0ALINC01608%0AAC004784.1%0AAL359091.1%0ASHTN1%0A4-Sep%0ADAAM2%0AKDM5D%0ANKAIN2%0ALPAR1%0ANAALADL2%0AZFYVE16%0ACAMK2N1%0AASPA%0ATMEFF2%0ADLC1%0ACIRBP%0APWRN1%0AMOBP%0APPFIA2%0AMOG%0AGRAMD2B%0AZDHHC9%0ASCD5%0AMOB3B%0AHHIP%0ACERCAM%0AMARCKSL1%0AANK3%0ATEKT4P2%0ADLG2%0AMAGI1%0ALINC01505%0AHECW2%0APPP1R14A%0ACREB5%0AAPLP1%0AUSP9Y%0APCBP4%0AGALNT13%0ASNX19%0ALANCL1%0AKIAA0930%0ATRIM2%0AGRIN2A%0APLXDC2%0APHLDB1%0ASEMA3B%0ARAPGEF5%0AFNBP1%0APOLR2F%0ANECAB1%0ASH3TC2-DT%0ABCAS1%0AANK2%0APSD3%0ACTNNA3%0ALMCD1-AS1%0ADANT2%0ASLC4A8%0APLEKHH1%0AARHGEF3%0AC5orf64%0ACMTM5%0ASH3D19%0APCDH9%0AZNF609%0AZEB2%0APTN%0ATMEM235%0AHEPACAM%0ACACNA2D1%0AERBB4%0ARPS4Y1%0ATXLNGY%0APTK2%0ASLC48A1%0ATMEM125%0ALSAMP%0APADI2%0ACOL9A3%0ATNS3%0AAL591684.2%0APLAC9P1%0AGMDS-DT%0AAL034405.1%0ALINC01006%0AHS3ST5%0ARGS11%0ARHOG%0AAC092683.1%0AOSTF1%0AGAPDH%0ATRIM59%0ASLC1A3%0ACOL18A1%0AASTN2%0AS100B%0ANINJ2%0ALINC01411%0AAATK%0ALINC01299%0AAPOD%0AEFS%0ACYP7B1%0AGAREM1%0ACNDP1%0ALINC01170%0ASRCIN1%0ACOL9A2%0AMAG%0ADOCK10%0ADEPTOR%0ANEDD4L%0AOTUD7A%0AATG10%0ATTTY10%0AMIR181A1HG%0ADOCK3%0AENOX2%0ATMEM178B%0ALIPA%0ALINC01877%0AKIAA1324L%0AENOSF1%0ASGCZ%0ALINC01102%0AAC098679.2%0AIPO13%0AAC004158.1%0AZNF536%0AFAM13C%0ANLGN4X%0ATMEM132C%0AFOLH1%0AFCHSD2%0APDE10A%0AGAB2%0ASLC6A1%0AMYO10%0AACTB%0AAC110285.1%0ASON%0ACPB2-AS1%0AAC027644.4%0ABORCS5%0AGALNT10%0AZFY%0ATTTY15%0AAPBB1%0AP2RX7%0ACD82%0ALAMA2%0AZNF462%0ALINC00639%0AADAM15%0ASERINC5%0AACYP2%0AAC138150.1%0APSEN1%0AMTMR7%0ASVEP1%0APLEKHG3%0AKCNAB1%0AMFGE8%0AROR1%0APLPPR1%0ASCFD2%0AAZIN2%0AKIF13A%0AFAHD2CP%0ATM7SF2%0ALINC00279%0ANRBP2%0AABTB2%0AGPR37%0ASREBF1%0AMAPK8IP1%0ATXNIP%0AFKTN%0ALINC01359%0AAC087463.4%0AFAM66D%0ACHRM5%0AAFMID%0ABCAS3%0AICA1L%0AKLHL42%0ALINC00609%0APTPRD%0AITFG1%0AMFSD6%0AKCNK1%0ASNX22%0ABABAM2%0ARAB6C-AS1%0AGPM6B%0AZMAT1%0AMIR325HG%0ASF1%0ATF%0ADNM1L%0AREPS2%0ACSRP1%0AARHGAP1%0AAL139383.1%0AAL354809.1%0AKIAA1755%0ACOL16A1%0ARAB11B%0ATARBP1%0AAUTS2%0ABDNF-AS%0AJMJD1C%0APDE4B%0AEXOC7%0AZMYM4%0APPM1H%0ASUCLG2%0AABCG1%0ARANBP17%0AEIF1AY%0APLLP%0AJAZF1%0ASEMA6A%0ACHADL%0AGNAI2%0AAC123912.1%0ARAP1GDS1%0ALINC01376%0ALCNL1%0ANALCN%0AAC114485.1%0ACXADR%0AWASHC3%0AFAM66B%0AAGPAT4%0AMEIS1%0ANENF%0AAC012593.1%0AMAGI2%0ATPPP3%0ASPTLC2%0ACDH1%0ATNS2%0ACR382287.2%0APRR5L%0ATPT1-AS1%0AFGFR2%0ACASTOR3%0ADIS3L2%0AANKS1B%0AAC090502.1%0AFP236383.1%0AKAT6B%0ATAPT1-AS1%0ARASGRF2%0ANTNG2%0ATMEM151A%0AGALNT6%0ADYSF%0ARHOU%0AELOVL1%0AZKSCAN1%0AFRMD4B%0APWRN3%0ASAMD12%0AZNF780B%0AKCNH8%0ATECPR2%0ALIMA1%0AACAP3%0AFTH1%0ASPARC%0ADPH6%0AINSYN2B%0AFKBP8%0AATG14%0AAC004925.1%0AAC093535.2%0ACOMMD1%0AFAM228B%0AEXOC6B%0ASH3GLB2%0ADOCK4%0AHIPK2%0AAC097103.2%0ACOL19A1%0ATMCC1%0APAQR6%0ABTBD9%0AVPS35L%0APIGU%0APEBP1%0ASEC14L5%0AEVI5%0ALANCL2%0AFTCDNL1%0ASHISA4%0AKLHL32%0ADENND5A%0APRKACB%0ATESK2%0ADDHD2%0AVWA1%0ACNKSR3%0ATPD52L2%0APVALEF%0APCNX2%0AMKRN3%0AIFNG-AS1%0AMAN2A2%0AFGD4%0ALUC7L2%0ALRRC4C%0AEPHB2%0AAC104041.1%0ADPYD%0AFHIT%0ARGN%0APARD3B%0AHCG23%0AFBXL2%0ASH3TC2%0ANEK3%0ALDAH%0ABACE1%0ANLGN3%0ADAPK2%0ACES4A%0ACOMMD10%0AAC009120.2%0ATRPM1%0AFBXO7%0ATAZ%0ACCDC149%0AKIF6%0ALIFR-AS1%0AHSPB1%0ASLC9A8%0AGJB1%0AMAP1A%0AAKAP6%0AKIAA0825%0AGPD1%0ABRSK1%0AFAAH%0ADCLK2%0APALM2%0ASCARB2%0ATBCB%0ARNF17%0AHPN-AS1%0AAL138689.2%0ACALN1%0ARNLS%0AARMC1%0AAC104151.1%0AAL359232.1%0AEPB41L4A%0ASUMF1%0AJPH1%0APOM121%0AAGAP9%0AAC087463.1%0AZNF337-AS1%0ATUBB4A%0ASNX29%0ATMEM135%0ADNAJB2%0AYEATS2-AS1%0AAP000282.1%0AMAML2%0APOU2AF1%0ATHEM4%0AIQCJ-SCHIP1%0AFAM135B%0AADGRA3%0AEPB41L3%0ATMEM123%0ASIPA1L1%0APACRG-AS1%0ATAGLN%0AMAP1B%0AAJAP1%0AFGGY%0ADNAH1%0ACUEDC1%0AAC242376.2%0AITPA%0ARETREG1%0ASHROOM1%0AAC005520.2%0ACDH11%0ARHBDL2%0APDE4D%0AARHGAP22%0APAXBP1-AS1%0AMBNL2%0AHAGLR%0APCYT1B%0ATFDP1%0ASPIDR%0ALINC00174%0AABCA1%0AHIP1%0AAPCDD1%0AGDI1%0AIMMP1L%0AOLIG2%0ARPL28%0AUSP4%0AAC006453.2%0ANECTIN3-AS1%0ACRYL1%0AZBTB20%0ARASSF4%0AAL365295.1%0ATMEM171%0ALHPP%0AKIFC3%0AAC004980.1%0AFTX%0APOLL%0ASTK31%0AELL2%0AAZIN1-AS1%0AAC007364.1%0ALZTS2%0ASLC16A1%0AMGC16275%0ATUFM%0AHLCS%0AKCNQ1OT1%0AMETRN%0ATFCP2%0APSPHP1%0ATJP1%0AGLTP%0AGRIA2%0ATASP1%0ASLCO3A1%0AEIF1B-AS1%0AACACA%0AFAM53B-AS1%0APIGN%0ANHLRC3%0ADCAF8%0AMAL%0AGPATCH2%0AOLIG1%0AAL589669.1%0ANRXN2%0ARAB30%0ATCF12%0AGPR37L1%0AGSN%0AEEFSEC%0AYBEY%0AGABPB2%0APTPRK%0ARCBTB1%0AFADS1%0ANECTIN1%0ARAB11B-AS1%0AHIBADH%0AIDNK%0AAC010145.1%0APTS%0AMPZL1%0AAC053527.1%0ASNX29P2%0ALARP6%0AEFHB%0APACRGL&ordered=true&all_results=false&no_iea=false&combined=false&measure_underrepresentation=false&domain_scope=annotated&significance_threshold_method=g_SCS&user_threshold=0.05&numeric_namespace=ENTREZGENE_ACC&sources=GO:MF,GO:CC,GO:BP,KEGG,TF,REAC,MIRNA,HPA,CORUM,HP,WP&background=&no_evidences=false

# Step 8
# PAGA analysis

By quantifying the connectivity of partitions (groups, clusters) of the single-cell graph, partition-based graph abstraction (PAGA) generates a much simpler abstracted graph (PAGA graph) of partitions, in which edge weights represent confidence in the presence of connections. By tresholding this confidence in paga(), a much simpler representation of the manifold data is obtained, which is nonetheless faithful to the topology of the manifold.

The confidence should be interpreted as the ratio of the actual versus the expected value of connetions under the null model of randomly connecting partitions. We do not provide a p-value as this null model does not precisely capture what one would consider “connected” in real data, hence it strongly overestimates the expected value. See an extensive discussion of this in [Wolf19].

### Diagnosis

In [ ]:
sc.tl.paga(adata, groups='leiden04')

In [ ]:
sc.pl.paga(adata, color=['leiden04', 'Diagnosis'])

In [ ]:
#Plot expression of 3 ranked group genes in the AD category

sc.pl.paga(adata, color=['PTPRM', 'ERBIN', 'NEAT1'], threshold = 0.03, layout="rt", root=4)

In [ ]:
#Plot expression of 3 ranked group genes in the control category

sc.pl.paga(adata, color=['PPP2R2B', 'UTY', 'NLGN4Y'], threshold = 0.03, layout="rt", root=4)

##### Recomputing the embedding using PAGA-initialization


In [ ]:
sc.tl.draw_graph(adata, init_pos='paga')

In [ ]:
sc.pl.draw_graph(adata, color=['Diagnosis'], legend_loc='on data')

In [ ]:
#AD
sc.pl.draw_graph(adata, color=['leiden04', 'PTPRM', 'ERBIN', 'NEAT1'], legend_loc='on data', ncols=2, vmin = 'p1.5', vmax = 'p99')

In [ ]:
#CONTROL
sc.pl.draw_graph(adata, color=['leiden04', 'PPP2R2B', 'UTY', 'NLGN4Y'], legend_loc='on data', ncols=2, vmin = 'p10', vmax = 'p99')


##### Reconstructing gene changes along PAGA paths for a given set of genes


In [ ]:
#Choose a root cell for diffusion pseudotime.
#AD

adata.uns['iroot'] = np.flatnonzero(adata.obs['Diagnosis']  == 'AD')[0]

In [ ]:
#Compute the pseudotime. After this step, some previous graphs can be adjusted. 

sc.tl.dpt(adata)

In [ ]:
#Root the PAGA graph according to the pseudotime

sc.pl.paga(adata, color=['Diagnosis'], layout="rt", root=4)

In [ ]:
gene_names = ['PTPRM', 'ERBIN', 'NEAT1',                      # AD
              'PPP2R2B', 'UTY', 'NLGN4Y']                    # Control

In [ ]:
#Use the fulll data for visualization:

sc.pl.draw_graph(adata, color=['leiden04', 'dpt_pseudotime'], legend_loc='on data',vmin = 'p1.5', vmax = 'p99')

In [ ]:
sc.pl.draw_graph(adata, color=['leiden04', 'Diagnosis'], legend_loc='on data')

In [ ]:
#AD is blue, C is orange

paths = [('AD', [1, 4, 8, 2]),
         ('Control', [0,3,5,6,7])]

In [ ]:
#Look at ordering according to prseudotime

_, axs = plt.subplots(ncols=2, figsize=(6, 2.5), gridspec_kw={'wspace': 0.05, 'left': 0.12})
plt.subplots_adjust(left=0.05, right=0.98, top=0.82, bottom=0.2)
for ipath, (descr, path) in enumerate(paths):
    _, data = sc.pl.paga_path(
        adata, path, gene_names,   #can add more genes
        show_node_names=False,
        ax=axs[ipath],
        ytick_fontsize=12,
        left_margin=0.15,
        n_avg=50,
        annotations=['dpt_pseudotime'],
        show_yticks=True if ipath==0 else False,
        show_colorbar=False,
        color_map='Greys',
        groups_key='leiden07',
        color_maps_annotations={'distance': 'viridis'},
        title='{} path'.format(descr),
        return_data=True,
        show=False)
#    data.to_csv('./write/paga_path_{}.csv'.format(descr))
#pl.savefig('./figures/paga_path_paul15.pdf')
plt.show()

# Step 9
# Save file

In [ ]:
adata.write("Oligo_Harmony.h5ad")